In [3]:
import os
import pandas as pd
import numpy as np
from sklearn.utils import class_weight
import tensorflow as tf
#import tensorflow_addons as tfa

In [13]:
img_size = (224, 224)

# train data
cwd = os.getcwd()
epochs = 100
df = pd.read_csv(cwd + '/state/train.csv')
X = list(df['image'][i] for i in range(len(df)))
y = list(df['emotion'][i] for i in range(len(df)))

In [5]:
weight = dict(enumerate(class_weight.compute_class_weight(class_weight='balanced', classes=np.unique(y), y=y)))
print(weight)

{0: 1.8591824644549764, 1: 2.264939376443418, 2: 1.8262918994413408, 3: 1.857421875, 4: 0.4784577387486279, 5: 0.41608771743742046, 6: 1.4797717842323652, 7: 1.3063186813186813}


In [10]:

def createModel():
    model = tf.keras.Sequential([
        tf.keras.layers.experimental.preprocessing.RandomRotation(
            0.05, fill_mode='nearest'
        ),

        tf.keras.applications.efficientnet_v2.EfficientNetV2B0(
            include_top=False, weights='imagenet', input_tensor=None,
            input_shape=(img_size[0], img_size[1], 3), include_preprocessing=True
        ),
        tf.keras.layers.Flatten(),


        tf.keras.layers.Dense(512, activation='relu'),

        tf.keras.layers.Dense(256, activation='relu'),

        tf.keras.layers.Dense(128, activation='relu'),

        tf.keras.layers.Dropout(0.5),

        tf.keras.layers.Dense(7, activation='softmax')
    ])
    model.compile(optimizer=tf.keras.optimizers.Adam(amsgrad=True),  # tf.keras.optimizers.Adam(amsgrad=True),  # tf.keras.optimizers.SGD(nesterov=True), # tfa.optimizers.RectifiedAdam(),
            loss=tf.keras.losses.SparseCategoricalCrossentropy(
                from_logits=True),
            metrics=['accuracy'])
    return model


In [14]:
es_callback = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy', patience=epochs*0.05, restore_best_weights=True)
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath="model/cp-{epoch:04d}.ckpt",
    verbose=0,
    save_weights_only=True,
    monitor='val_accuracy',
    save_best_only=True)
tb_callback = tf.keras.callbacks.TensorBoard(
    log_dir="./logs",
            update_freq="epoch")

In [15]:
model = createModel()


24289280/24274472 [==============================] - 5s 0us/step
